In [1]:
import pandas as pd
%matplotlib inline
import tangos
import tangos.examples.mergers as mergers

import numpy as np
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
h = 0.6776942783267969

from tangos_halo_module.halo_properties import track_halo_property, get_main_progenitor_branch, get_timesteps, ID_to_sim_halo_snap, infall_final_n_particles, infall_final_coordinates, apocentric_distance, disruption_time, accretion_time, orbit_interpolation, infall_velocity, quenching_time, max_sSFR_time, max_mass_time 
from tangos_halo_module.path import get_file_path, get_halo_snap_num, read_file
from tangos_halo_module.halos import ID_to_tangos_halo, get_survivors, get_zombies, get_host, get_survivor_IDs, get_zombie_IDs, blockPrint, enablePrint, tangos_to_pynbody_halo

In [2]:
ids = np.loadtxt('ids100.txt')
ids=[int(i) for i in ids]
print(len(ids))
ids

194


[148409610,
 148409610270,
 14840961090,
 14840961101,
 148409612,
 14840961292,
 148409613,
 1484096189,
 14840962,
 1484096217,
 1484096223,
 14840962297,
 148409623,
 1484096263,
 1484096290,
 14840963,
 1484096306,
 148409636,
 1484096380,
 148409639,
 14840964,
 1484096438,
 148409644,
 148409648,
 1484096563,
 148409662,
 148409669,
 1484096779,
 1484096932,
 1480225104,
 148027562,
 1480275869,
 1480347728,
 14803478,
 1480384109,
 1480384115,
 1480384124,
 1480384163,
 14803846,
 1480384990,
 1480456173,
 1480456309,
 148045663,
 1480512264,
 14805124,
 148051272,
 148051285,
 148051289,
 148064011,
 14807012,
 148070125,
 148070149,
 148070165,
 148089619,
 148097431,
 148102421,
 148102429,
 14810249,
 14811066,
 148115210,
 14812803,
 14812806,
 148153610,
 148153614,
 148192016,
 14821761398,
 14824322,
 14829442,
 229409610,
 22940961327,
 22940961381,
 229409618,
 2294096191,
 229409624,
 22940963661,
 229409645,
 229409646,
 229409648,
 229409659,
 229409667,
 229409689,

tang_halo = ID_to_tangos_halo(ID='14840961101', resolution=100)
get_main_progenitor_branch(tangos_halo=tang_halo, simulation='h148', resolution=100)

tang_halo = ID_to_tangos_halo(ID='14840961101', resolution=100)
get_main_progenitor_branch(tangos_halo=tang_halo, simulation='h148', resolution=100)

In [3]:
def sSFRtrack(ID, simulation, resolution):           
    tang_halo = ID_to_tangos_halo(ID=ID, resolution=resolution)
    print(tang_halo)
    pyn_halo = tangos_to_pynbody_halo(tangos_halo=tang_halo, simulation=simulation, resolution=resolution)
    mass = np.asarray(pyn_halo.s['mass'].in_units('Msol'))
    age = pyn_halo.properties['time'].in_units('Gyr') - pyn_halo.s['tform'].in_units('Gyr') #SimArray in Gyr
    new_mass = np.sum(mass[age <= 0.1]) # Formed within last 100 Myr
    total_mass = np.sum(mass)
    SFR = new_mass/1e8 # Msol per year
    sSFR = SFR/total_mass
    return SFR, sSFR, total_mass, new_mass

In [5]:
def sSFR_track(ID=0, simulation=0, status=0, tangos_halo=0, halo_id=0, snap_num=0, resolution=1000):
    
#     from tangos_halo_module.halos import tangos_to_pynbody_halo, get_main_progenitor_branch

    if ID != 0:
        print('Started ', ID)
        simulation, status, halo_id, snap_num = ID_to_sim_halo_snap(ID=ID)
        print(simulation, status, halo_id, snap_num)
        tangos_halo = ID_to_tangos_halo(ID=ID, resolution=resolution)
    
    # Satellite MPB ids
    MPB, MPB_ids = get_main_progenitor_branch(tangos_halo=tangos_halo, simulation=simulation, resolution=resolution)
    print(MPB_ids)
    print(MPB)
    # Host MPB and ids
    host = get_host(simulation=simulation, resolution=resolution)
    print(host)
    host_MPB, host_ids = get_main_progenitor_branch(tangos_halo=host, simulation=simulation, resolution=resolution)
    
    # Timestep extensions
    timesteps = get_timesteps(simulation=simulation, resolution=resolution)[3]
    print(timesteps)
    # Only consider halo while separate from host
    MPB_ids[MPB_ids==host_ids] = 0
    print(MPB_ids)

    for i in range(len(MPB_ids)):
        if MPB_ids[i]!=0:
            if len(str(MPB[i]))>6:
                #########################################################################
                MPB_ids[i]=get_halo_snap_num(tangos_halo=MPB[i])[2]
            else: 
                idx=MPB_ids[i]
                idk=str(simulation)[1:]+str(timesteps[i])+str(idx)
                MPB_ids[i]=idk
                
    print('MPB new IDs: ', MPB_ids)
    IDs_track = np.zeros(0)
    SFR_track = np.zeros(0)
    sSFR_track = np.zeros(0)
    mass_track = np.zeros(0)
    new_mass_track = np.zeros(0)

    for index, idx in enumerate(MPB_ids):
        print(idx)
        if idx==0:
            IDs_track = np.concatenate((IDs_track, [-1]), axis=None)
            SFR_track = np.concatenate((SFR_track, [-1]), axis=None)
            sSFR_track = np.concatenate((sSFR_track, [-1]), axis=None)
            mass_track = np.concatenate((mass_track, [-1]), axis=None)
            new_mass_track = np.concatenate((new_mass_track, [-1]), axis=None)
#         elif len(str(MPB[index]))<6: #None case
# #             ID_to_tangos_halo(ID=ID, resolution=resolution)
#             SFR, sSFR, total_mass, new_mass = sSFRtrack(ID=idx, simulation=simulation, resolution=resolution)
#             IDs_track = np.concatenate((IDs_track, [idx]), axis=None)
#             SFR_track = np.concatenate((SFR_track, [SFR]), axis=None)
#             sSFR_track = np.concatenate((sSFR_track, [sSFR]), axis=None)
#             mass_track = np.concatenate((mass_track, [total_mass]), axis=None)
#             new_mass_track = np.concatenate((new_mass_track, [new_mass]), axis=None)
        else:
            SFR, sSFR, total_mass, new_mass = sSFRtrack(ID=idx, simulation=simulation, resolution=resolution)
            IDs_track = np.concatenate((IDs_track, [idx]), axis=None)
            
            SFR_track = np.concatenate((SFR_track, [SFR]), axis=None)
            sSFR_track = np.concatenate((sSFR_track, [sSFR]), axis=None)
            mass_track = np.concatenate((mass_track, [total_mass]), axis=None)
            new_mass_track = np.concatenate((new_mass_track, [new_mass]), axis=None)
    print(IDs_track, SFR_track, sSFR_track, mass_track, new_mass_track)
    dat = {'ID': ID,
           'IDs': [IDs_track],
           'SFR': [SFR_track], 
           'sSFR': [sSFR_track], 
           'stellar_mass': [mass_track], 
           'new_stellar_mass': [new_mass_track],}
    df = pd.DataFrame(data=dat)
    return df
    
df = pd.read_csv('sSFR_data100.csv')
completed_ids = df['ID'].to_numpy()
len(completed_ids)

194

In [ ]:
sSFR_track(ID=148409639, resolution=100)
ID_to_tangos_halo(ID=148409639, resolution=100)

In [ ]:
count=0
while True:
    try:
        print('Watch me go for the ', count, 'th time...')
        for idx in ids:
            if idx in completed_ids:
                print('Already Exists ', idx)
                pass
            else:
                print('Started ', idx)
                sat_df = sSFR_track(ID=idx, resolution=100)
                df = pd.concat([df, sat_df])
                df.to_csv('sSFR_data100.csv', index=False)
                completed_ids = np.concatenate((completed_ids, [idx]), axis=None)
                print('Finished ', idx)
                print('Completed Count: ', len(completed_ids))
    except: 
        count+=1
        continue

In [13]:
mass = np.asarray(pyn_halo.s['mass'].in_units('Msol'))

In [14]:
mass

array([4912.04043909, 4920.29554547, 4912.04475662, 4915.48064242,
       4912.04907414, 4912.07929681, 4912.08102382, 4919.80205245,
       4923.73229478, 4920.88316052, 4922.04889207, 4912.09052238,
       4918.86558144, 4922.34162022, 4912.09527165, 4922.92750826,
       4923.29838359, 4912.10908773, 4914.82610574, 4921.5204271 ])